In [1]:
from sympy import *
from sympy.solvers.solveset import linsolve
x,y,z,a,b,c = symbols('x y z a b c')
AB, AC, ABx, ABy, ACx, ACy = symbols('AB AC AB_x AB_y AC_x AC_y')
A,B,C,D,T = symbols('A B C D T')
Ax, Ay, Bx, By, Cx, Cy, Dx, Dy, Tx, Ty = symbols('A_x A_y B_x B_y C_x C_y D_x D_y T_x T_y')

### Basis Transformation for interpolation

![Basis Transformation for interpolation](img/basis-transform-interpolation.svg)


$\vec{A}$, $\vec{B}$ and $\vec{C}$ are points on the screen of arrived rays. $\vec{T}$ is a sample point, whose intensity value needs to be interpolated from the ray intensities:
$$ I(\vec{T}) = I(\vec{A}) \;+\; b_A\cdot \left(I(\vec{B}) - I(\vec{A})\right) \;+\; c_A\cdot \left( I(\vec{C}) - I(\vec{A}) \right) $$

within the triangle: $b_A \geq 0$, $c_A\geq 0$ and $(b_A+c_A) < 1$

Furthermore:
$$ \vec{AB} = \vec{B} - \vec{A} $$
$$ \vec{AC} = \vec{C} - \vec{A} $$

$$ \vec{T} = \vec{A} + b_A\vec{AB} + c_A\vec{AC} $$

The same method can be applied for the points inside $\vec{D}$, $\vec{B}$ and $\vec{C}$, 

with $b_D>0$, $c_D>0$ and $(b_D+c_D)\leq 1$, to cover the other half of the quadrangle.

### Solution for b and c:

In [2]:
solutions = linsolve([Ax + b*ABx + c*ACx - Tx,\
                           Ay + b*ABy + c*ACy - Ty], (b, c))
bA = list(solutions)[0][0]
cA = list(solutions)[0][1]
solutions

FiniteSet(((AC_x*A_y - AC_x*T_y - AC_y*A_x + AC_y*T_x)/(AB_x*AC_y - AB_y*AC_x), (-AB_x*A_y + AB_x*T_y + AB_y*A_x - AB_y*T_x)/(AB_x*AC_y - AB_y*AC_x)))

In [3]:
bA_expr = collect(bA, (ACx, ACy))
bA_expr

(AC_x*(A_y - T_y) + AC_y*(-A_x + T_x))/(AB_x*AC_y - AB_y*AC_x)

In [6]:
cA_expr = collect(cA, (ABx, ABy))
cA_expr

(AB_x*(-A_y + T_y) + AB_y*(A_x - T_x))/(AB_x*AC_y - AB_y*AC_x)

So, solutions are:
$$ \mathrm{denom} = AB_x AC_y - AB_y AC_x $$

$$ b_A = \frac{(T_x-A_x) AC_y + (A_y-T_y) AC_x}{\mathrm{denom}}$$

$$ c_A = \frac{(A_x-T_x)AB_y + (T_y-A_y)AB_x}{\mathrm{denom}}$$

### Further simplification
These expressions contain the following pattern multiple times:

$$\mathrm{Area\_Para}(\vec{v}, \vec{w}) = v_x w_y - v_y w_x$$

which computes the area of the parallelogram with 2D vectors $\vec{v}$ and $\vec{w}$ as sides. Furthermore $\vec{AT} = \vec{T} - \vec{A}$. The expressions can then be written as:
$$ \mathrm{denom} = \mathrm{Area\_Para}(\vec{AB}, \vec{AC})$$

$$ b_A = \frac{\mathrm{Area\_Para}(\vec{AT}, \vec{AC})}{\mathrm{denom}}$$

$$ c_A = \frac{\mathrm{Area\_Para}(-\vec{AT}, \vec{AB})}{\mathrm{denom}}$$

In [21]:
# Check that the new expressions are indeed equivalent
def area_para(v, w):
    """Area of parallelogram."""
    return v[0]*w[1] - v[1]*w[0]

# Define vectors
AB = (ABx, ABy)
AC = (ACx, ACy)
AT = (Tx-Ax, Ty-Ay)
TA = (Ax-Tx, Ay-Ty)

# New expressions
denom = area_para(AB, AC)
bA_newexpr = area_para(AT, AC) / denom
cA_newexpr = area_para(TA, AB) / denom

In [22]:
simplify(Eq(bA_expr, bA_newexpr))

True

In [23]:
simplify(Eq(cA_expr, cA_newexpr))

True